In [1]:
from qiskit import Aer
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import AerPauliExpectation, PauliSumOp
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SPSA
import numpy as np

In [5]:
def run_vqe(ansatz, hamiltonian, maxiter=1000, vocal=False):
    # Choose the backend
    backend = Aer.get_backend('statevector_simulator')

    # Choose the optimizer
    optimizer = SPSA(maxiter=maxiter)

    # Choose the expectation value calculator
    expectation = AerPauliExpectation()

    # Create the VQE algorithm instance
    vqe = VQE(ansatz=ansatz,
            optimizer=optimizer,
            quantum_instance=backend,
            expectation=expectation)

    # Run the VQE optimization
    result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)
    # Get the optimal energy and parameters
    optimal_energy = result.optimal_value
    optimal_params = result.optimal_point

    # Print the results
    if vocal:
        print("Optimal Energy:", optimal_energy)
        print("Optimal Parameters:", optimal_params)

    return optimal_energy, optimal_params

Først $N = 2, (J = 1)$ tilfelle. Vi setter opp $3 \times 3$ matrisen i $\vert J, J_z \rangle$ og finner laveste egenverdi. Deretter setter vi opp Hamiltonen med Pauli matriser og kjører VQE

In [6]:
# Setup Lipkin Hamiltonian in J scheme with N = 2
eps, V = 1, 1.1

H_exact = np.zeros((3,3))
H_exact[0,0] = -eps
H_exact[2,2] = eps
H_exact[2,0] = H_exact[0,2] = -V

E, C = np.linalg.eigh(H_exact)
print(f"J = 2 matrix, E/eps = {E[0]/eps}")
v = V/eps
print(f"Closed form (eq. 24) {-np.sqrt(eps**2 + V**2)}")

J = 2 matrix, E/eps = -1.4866068747318506
Closed form (eq. 24) -1.4866068747318506


In [7]:
# Define the ansatz circuit
ansatz = RealAmplitudes(num_qubits=2, entanglement='linear', reps=1)

# J = 2 lipkin hamiltonian (eq. 62)
hamiltonian = PauliSumOp.from_list([
    ("ZI", 0.5),
    ("IZ", 0.5),
    ("XX", -v/2),
    ("YY", v/2)
])

_, _= run_vqe(ansatz, hamiltonian, maxiter=1000,vocal=True)

Optimal Energy: -1.4866058584212714
Optimal Parameters: [1.27668105 0.68825168 2.015049   1.90933592]


Så $N = 4, (J = 2)$ tilfelle. Vi setter opp $5 \times 5$ matrisen i $\vert J, J_z \rangle$ og finner laveste egenverdi. Deretter setter vi opp Hamiltonen med Pauli matriser og kjører VQE

In [74]:
# Setup Lipkin Hamiltonian in J scheme with N = 4
eps, V = 2, 0.5

H_exact = np.zeros((5,5))
H_exact[0,0] = -2*eps
H_exact[1,1] = -eps
H_exact[3,3] = eps
H_exact[4,4] = 2*eps

H_exact[2,0] = H_exact[0,2] = H_exact[2,4] = H_exact[4,2] = -np.sqrt(6)*V
H_exact[3,1] = H_exact[1,3] = -3*V

E, C = np.linalg.eigh(H_exact)
print(f"J = 2 matrix, E/eps = {E[0]/eps}")
v = V/eps
print(f"{v = }, closed form (eq 63) E = {-2*np.sqrt(3*v**2+1)}")

J = 2 matrix, E/eps = -2.1794494717703365
v = 0.25, closed form (eq 63) E = -2.179449471770337


In [75]:
# Setup Lipkin Hamiltonian in J scheme with N = 4
eps, V = 2, 0.5

H_exact = np.zeros((5,5))
H_exact[0,0] = -2*eps
H_exact[1,1] = -eps
H_exact[3,3] = eps
H_exact[4,4] = 2*eps

H_exact[2,0] = H_exact[0,2] = H_exact[2,4] = H_exact[4,2] = -np.sqrt(6)*V
H_exact[3,1] = H_exact[1,3] = -3*V

E, C = np.linalg.eigh(H_exact)
print(f"J = 2 matrix, E/eps = {E[0]/eps}")
v = V/eps
print(f"{v = }, closed form (eq 63) E = {-2*np.sqrt(3*v**2+1)}")

J = 2 matrix, E/eps = -2.1794494717703365
v = 0.25, closed form (eq 63) E = -2.179449471770337


In [76]:
# Define the ansatz circuit
ansatz = RealAmplitudes(num_qubits=2, entanglement='linear', reps=1)
v_prime = v * np.sqrt(6)/2

# J = 2 lipkin hamiltonian (eq. 62)
hamiltonian = PauliSumOp.from_list([
    ("ZI", -1),
    ("IZ", -1),
    ("XI", -v_prime),
    ("IX", -v_prime),
    ("ZX", -v_prime),
    ("XZ", v_prime)
])

_, _= run_vqe(ansatz, hamiltonian, maxiter=1000,vocal=True)

Optimal Energy: -2.1794443964385635
Optimal Parameters: [ 3.04077711  0.68710579 -2.5086088  -2.42707604]
